# Create distributed cylinders
This note book uses MCMRSimulator v0.9.0 and custom function `repel_distributed_radius()` in `repel_cylinders.jl` to generate parallel cylinder substrates with Gamma-distributed diameters for our simulation. The custom function is needed because MCMRSimulator's built-in function `random_positions_radii()` generated cylinder packing configurations that were highly inhomogeneous, which resulted in clusters of cylinders and large gaps between clusters. This affected the consistency and accuracy of our simulated signal. `repel_distributed_radius()` takes the output of `random_positions_radii()` and add repulsions between cylinders to create a more homogeneous packing configuration. It returns the positions and radius of all cylinders in a block. The `Cylinders()` constructor then creates the cylinder object in the simulation with the given position and radius and infinitely repeats the block to fill in all space. In later versions of MCMRSimulator, such as the v0.11.0 we used for the example simulation, the custom function has been integrated into the `random_positions_radii()`, therefore using `random_positions_radii()` alone is sufficient.

In [2]:
# Load the pkg module to activate the correct environment
using Pkg
Pkg.activate("../julia-envs/old_mcmr")
Pkg.instantiate()
# import the relevant packages
using MCMRSimulator
using Printf
include("../repel_cylinders.jl")

  Activating project at `~/Library/CloudStorage/OneDrive-Nexus365/projects/Papers/2024 MRM/gitrepo/MT-and-permeability-effect-on-two-compartment-dMRI-WM-model/julia-envs/old_mcmr`
Precompiling project...
  14511.8 ms  ✓ ArrayInterface → ArrayInterfaceBandedMatricesExt
  16894.1 ms  ✓ PNGFiles
  16896.2 ms  ✓ Sixel
  17309.8 ms  ✓ ImageAxes
   1408.7 ms  ✓ FiniteDiff → FiniteDiffBandedMatricesExt
   3521.5 ms  ✓ WebP
   1431.8 ms  ✓ ImageMetadata
   8098.1 ms  ✓ Roots → RootsForwardDiffExt
   1901.6 ms  ✓ Netpbm
   5921.5 ms  ✓ FiniteDiff → FiniteDiffStaticArraysExt
  25133.7 ms  ✓ JpegTurbo
    960.4 ms  ✓ NLSolversBase
  10124.3 ms  ✓ Distributions → DistributionsTestExt
   3085.5 ms  ✓ LineSearches
   2398.9 ms  ✓ Optim
  99886.1 ms  ✓ Makie
  24961.6 ms  ✓ CairoMakie
  83042.4 ms  ✓ MCMRSimulator
  18 dependencies successfully precompiled in 223 seconds. 333 already precompiled.
[ Info: Precompiling MCMRSimulator [eac06e4d-a854-4bf6-b44f-54c98b0401d1] (cache misses: wrong dep versio

repel_distributed_radius

In [3]:
T2s = 10:10:300       # Define effective T2 (MT strength) range (ms)
T2_bound=1e-2         # Bound pool T2 to model surface relaxivity (ms)
rho=0.65              # Cylinder volume density
r_mean = 2            # set the mean cylinder radius 
r_var = 1             # set the variance of cylinder radius
rep = r_mean*100          # (half of) the distance over which the substrate repeats itself spatially
t_dwell = 30          # Dwell time of isochromats in the bound pool, controls the strength of surface relaxivity
svratio = 4*rho/r_mean             # Calculate the surface to volume ratio that is used to calculate the surface density later

# Make directories to store cylinders
mkpath("MT")
# mkpath("perm_cyl")

# Create a substrate with parallel cylinders of distributed diameters
res = MCMRSimulator.random_positions_radii([rep, rep], rho, 2, mean=r_mean, variance=r_var)            # res contains two lists [1] is the positions of the cylinder centres (circle centres in 2d), [2] is the radii of all cylinders
repeled = repel_distributed_radius(res[2], res[1], rep, maxiter=5000, repulsion_strength=1e-3)
for T2 = T2s                               # Loop through the desired MT range
    surf_dens = 1/svratio*t_dwell/T2       # Surface density of isochromats that achieves the given T2, it's the ratio of isochromat density on the substrate (cylinder) and isochromat density in the volume of interest
    rng_geom = Cylinders(position=res[1], radius=res[2], dwell_time=t_dwell, density=surf_dens, repeats=[rep, rep], R2_surface=1/T2_bound)                     # Create the cylinder substrate object
    MCMRSimulator.write_geometry("./MT/cylinders_MT_"* string(T2) *"_sus_0_perm_0.000_rmean_"*@sprintf("%.2f",r_mean)*"_density_0.65.json", rng_geom)      # Save the substrate object
end
# for perm in 0:0.001:0.02                   # Loop through the desired permeability range
#     rng_geom = Cylinders(position=res[1], radius=res[2], permeability=perm, repeats=[rep, rep])                                                                       # Create the cylinder substrate object 
#     MCMRSimulator.write_geometry("./perm_cyl/cylinders_MT_0_sus_0_perm_"*@sprintf("%.3f",perm)*"_rmean_"*@sprintf("%.2f",r_mean)*"_density_0.65.json", rng_geom)      # Save the substrate object      
# end

There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values for Field(position)
There are 1627 Cylinder based on values 